In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
import asyncio
import base64
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content, Attachment, FileContent, FileName, FileType, Disposition



In [3]:

load_dotenv(override=True)


True

In [4]:
attractions_planner_agent_instructions = """
You are a attractions planner. You are given a destination and need to plan all must-do things in that city.
You should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.
Attractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.
"""

activity_planner_agent_instructions = """
You are a activity planner. You are given a destination 
You need to find all local best activities in that city with their popularity.
Activities should be something to experience like skydiving, kayaking, sunset sails, snorkeling, etc.
"""

restaurant_planner_agent_instructions = """
You are a restaurant planner. You are given a destination and number of days to spend.
You need to find all local best restaurants in that city with their ratings and reviews. 
You need to plan for 3 meals a day for the number of days you are staying.
"""

attractions_planner_agent = Agent(
    name="Attractions Planner",
    instructions = attractions_planner_agent_instructions,
)

activity_planner_agent = Agent(
    name="Activity Planner",
    instructions = activity_planner_agent_instructions,
)

restaurant_planner_agent = Agent(
    name="Restaurant Planner",
    instructions = restaurant_planner_agent_instructions,
)



In [5]:
itenary_planner_agent_instructions = """
    You are given a list of attractions, activities and restaurants. 
    You need to pick best attractions, activities and restaurants out of the list to plan a trip to the destination.
    You give a day by day plan of the trip with timings for various attractions and activities.
    You'll also plan for appropriate schedules for 3 meals a day in nearby restaurants to the current schedule.
"""

itenary_planner_agent = Agent(
    name="Itenary Planner",
    instructions=itenary_planner_agent_instructions,
)

In [6]:
calender_generator_agent_instructions = """
You are a calender generator. You are given a ordered list of attractions, activities and restaurants with appropriate days and timings.
You need to generate a single ics file with all the events and their details.
You need to include all the details like name, description, start and end time, location, etc.
This ics file should be in a format that can be imported into any calendar app.
"""

calender_generator_agent = Agent(
    name="Calender Generator",
    instructions = calender_generator_agent_instructions,
)

In [7]:
activity_planner_tool = activity_planner_agent.as_tool(tool_name="activity_planner_tool", tool_description="Plan only activities for the given destination and number of days")
attractions_planner_tool = attractions_planner_agent.as_tool(tool_name="attractions_planner_tool", tool_description="Plan only attractions for the given destination and number of days")
restaurant_planner_tool = restaurant_planner_agent.as_tool(tool_name="restaurant_planner_tool", tool_description="Plan only restaurants for the given destination and number of days")
itenary_planner_tool = itenary_planner_agent.as_tool(tool_name="itenary_planner_tool", tool_description="Plan a trip to the given destination and number of days by calling activity planner, attractions planner and restaurant planner tools")
calender_generator_tool = calender_generator_agent.as_tool(tool_name="calender_generator_tool", tool_description="Generate a calender with all the events and their details")


In [8]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [10]:
@function_tool
def send_html_email(subject: str, html_body: str, ics_data: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("harishjmhss@gmail.com")  # Change to your verified sender
    to_email = To("harishneyveli96@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)

    # Create attachment
    encoded_ics_data = base64.b64encode(ics_data.encode()).decode()
    attached_file = Attachment()
    attached_file.file_content = FileContent(encoded_ics_data)
    attached_file.file_type = FileType("text/calendar")
    attached_file.file_name = FileName("meeting.ics")
    attached_file.disposition = Disposition("attachment")

    # Create mail
    mail = Mail(from_email, to_email, subject, content)
    mail.add_attachment(attached_file)
    response = sg.client.mail.send.post(request_body=mail.get())

    return {"status": "success"}

In [12]:
emailer_agent_tools = [subject_tool, html_tool, send_html_email]
emailer_agent_tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f7a37cfaa20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f7a37cfade0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sale

In [13]:
instructions ="""You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the "send_html_email" tool to send the email with the subject and HTML body.( you're often missing this tep, please make sure, email is sent using "send_html_email" tool)
Dont ask if you need to send the email, just send it.
Also attach the calender appointment as a downloadable ics file.
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=emailer_agent_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [14]:
travel_planner_agent_tools = [
    activity_planner_tool,
    attractions_planner_tool,
    restaurant_planner_tool,
    calender_generator_tool
]


In [15]:
travel_planner_agent_instructions = """
    You are a travel planner. You are given a particular destination and number of days to spend. 
    You need to use the tools given to you to plan the trip.
    You need to use the tools in a way that is most efficient and effective.
    Send your output to the emailer_agent.
"""

travel_planner_agent = Agent(
    name="travel_planner_agent",
    instructions = travel_planner_agent_instructions,
    tools = travel_planner_agent_tools,
    handoffs = [emailer_agent],
    model = "gpt-4o-mini"
)



In [16]:
message = "New York, 5 days"

with trace("Trip Planner"):
    result = await Runner.run(travel_planner_agent, message)